<h2>importing the libaries


In [37]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow.lite as tflite
import json
from flask import Flask, request, jsonify

<h2>Load the dataset

In [38]:
df = pd.read_csv("smart_grid_dataset.csv").dropna()  # Replace with actual dataset path
df.head()

,Timestamp,Voltage (V),Current (A),Power Consumption (kW),Reactive Power (kVAR),Power Factor,Solar Power (kW),Wind Power (kW),Grid Supply (kW),Voltage Fluctuation (%),Overload Condition,Transformer Fault,Temperature (°C),Humidity (%),Electricity Price (USD/kWh),Predicted Load (kW)
0,2024-01-01 00:00:00,232.483571,5.124673,1.191402,0.399029,0.907766,46.614845,26.399566,0.0,1.055937,0,0,17.842767,64.523353,0.416156,1.233158
1,2024-01-01 00:15:00,229.308678,22.213519,5.093753,1.248030,0.831813,6.907170,18.555011,0.0,-3.910902,1,0,18.753698,49.671940,0.492334,5.247824
2,2024-01-01 00:30:00,233.238443,46.132217,10.759806,2.790464,0.999005,15.477640,12.905025,0.0,-0.197551,0,0,16.719325,48.472769,0.427785,10.580278
3,2024-01-01 00:45:00,237.615149,47.648972,11.322118,1.956965,0.896462,35.746398,28.937129,0.0,2.671921,0,0,15.364564,75.766776,0.487953,11.225946
4,2024-01-01 01:00:00,228.829233,7.410166,1.695663,0.624718,0.870681,27.555038,14.184648,0.0,3.826937,0,0,38.760333,61.384213,0.290078,1.696639


<h2>Preprocessing</h2>

In [39]:
df['Hour'] = pd.to_datetime(df['Timestamp']).dt.hour  # Extract hour from timestamp
df = df[['Voltage (V)', 'Current (A)', 'Power Consumption (kW)', 'Hour']]

X = df  # No need to drop anything
y = (df['Power Consumption (kW)'] > df['Power Consumption (kW)'].quantile(0.75)).astype(int)  
# Peak hour = Top 25% power usage

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

<h2>test train spilt

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

<h2>Train the model

In [42]:
model = keras.Sequential([
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),  # L2 Regularization
    layers.Dropout(0.3),  # Dropout to prevent overfitting
    layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
1250/1250 [==============================] - 9s 5ms/step - loss: 0.3127 - accuracy: 0.9296 - val_loss: 0.1631 - val_accuracy: 0.9891
Epoch 2/50
1250/1250 [==============================] - 5s 4ms/step - loss: 0.1571 - accuracy: 0.9734 - val_loss: 0.1375 - val_accuracy: 0.9758
Epoch 3/50
1250/1250 [==============================] - 5s 4ms/step - loss: 0.1339 - accuracy: 0.9779 - val_loss: 0.1155 - val_accuracy: 0.9932
Epoch 4/50
1250/1250 [==============================] - 6s 5ms/step - loss: 0.1198 - accuracy: 0.9813 - val_loss: 0.1045 - val_accuracy: 0.9957
Epoch 5/50
1250/1250 [==============================] - 5s 4ms/step - loss: 0.1102 - accuracy: 0.9823 - val_loss: 0.0987 - val_accuracy: 0.9885
Epoch 6/50
1250/1250 [==============================] - 5s 4ms/step - loss: 0.1020 - accuracy: 0.9851 - val_loss: 0.0896 - val_accuracy: 0.9984
Epoch 7/50
1250/1250 [==============================] - 5s 4ms/step - loss: 0.0954 - accuracy: 0.9852 - val_loss: 0.0842 - val_accuracy:

<h2>convert to tflite

In [30]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("peak_hour_model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\kesha\AppData\Local\Temp\tmp0x70jhuh\assets


INFO:tensorflow:Assets written to: C:\Users\kesha\AppData\Local\Temp\tmp0x70jhuh\assets


<h2>Validate the model

In [31]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

313/313 [==============================] - 1s 3ms/step - loss: 0.0104 - accuracy: 0.9955
Test Accuracy: 0.9955


Your model is performing exceptionally well, achieving **100% accuracy on both the training and validation sets**. However, this might indicate **overfitting** rather than actual generalization. Here’s what you can do:

### 🔍 **Possible Issues:**
1. **Overfitting**  
   - The loss and accuracy suggest that your model has memorized the training data rather than learning general patterns.
2. **Data Imbalance**  
   - If one class (peak vs. non-peak) dominates, the model may just predict the majority class.
3. **Model Simplicity**  
   - If the problem is simple, a deep learning model might be overkill. Try logistic regression or decision trees.

---

### 🛠 **How to Improve the Model:**
1. **Introduce Regularization:**
   - Add **Dropout** layers (e.g., `Dropout(0.3)`)
   - Use **L2 regularization** (`kernel_regularizer=l2(0.001)`)
2. **Augment Data or Balance Classes:**
   - Check if the dataset is balanced. If not, try **SMOTE (Synthetic Minority Over-sampling Technique)**.
3. **Reduce Model Complexity:**
   - Use fewer layers or fewer neurons per layer.
4. **Early Stopping:**  
   - Stop training when validation loss stops decreasing.
   ```python
   from tensorflow.keras.callbacks import EarlyStopping
   early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
   ```
5. **Try Simpler Models:**  
   - A decision tree or random forest could work well for structured data.

Would you like me to modify the model and rerun it? 🚀

Better Alternative (Let Model Learn Peak Hours)
Instead of defining "Peak_Hour" manually, train a model without explicitly adding it:

df['Hour'] = pd.to_datetime(df['Timestamp']).dt.hour  # Extract hour from timestamp
df = df[['Voltage (V)', 'Current (A)', 'Power Consumption (kW)', 'Hour']]

X = df  # No need to drop anything
y = (df['Power Consumption (kW)'] > df['Power Consumption (kW)'].quantile(0.75)).astype(int)  
# Peak hour = Top 25% power usage

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
